# MiniLisp Grammar LL(1) Analysis

This notebook analyses the MiniLisp grammar to verify it satisfies the LL(1) property.

Confirm predictive parsing:
* [x] FIRST set
* [x] FOLLOW set
* [x] Parse table

---

**Grammar:**
```
<program> ::= <expr>
<expr> ::= NUMBER
| IDENTIFIER
| '(' <paren-expr> ')'
<paren-expr> ::= '+' <expr> <expr>
| '×' <expr> <expr>
| '=' <expr> <expr>
| '−' <expr> <expr>
| '?' <expr> <expr> <expr>
| 'λ' IDENTIFIER <expr>
| '≜' IDENTIFIER <expr> <expr>
| <expr> <expr>*
```

In [3]:
grammar = {
    '<program>': [['<expr>']],
    '<expr>': [['NUMBER'], ['IDENTIFIER'], ['(', '<parent-expr>', ')']],
    '<parent-expr>': [
        ['+', '<expr>', '<expr>'],
        ['×', '<expr>', '<expr>'],
        ['=', '<expr>', '<expr>'],
        ['-', '<expr>', '<expr>'],
        ['?', '<expr>', '<expr>', '<expr>'],
        ['λ', 'IDENTIFIER', '<expr>'],
        ['≜', 'IDENTIFIER', '<expr>', '<expr>'],
        ['<expr>', '<expr>*']
    ]
}

terminals = {'NUMBER', 'IDENTIFIER', '+', '×', '=', '-', '?', 'λ', '≜', '(', ')', '$'}
non_terminals = set(grammar.keys())

In [4]:
FIRST = {nt: set() for nt in non_terminals}

changed = True

while changed:
    changed = False

    for nt in non_terminals:
        for production in grammar[nt]:
            can_be_empty = True

            for token in production:
                if token in terminals:
                    before = len(FIRST[nt])

                    FIRST[nt].add(token)

                    if len(FIRST[nt]) != before:
                        changed = True

                    can_be_empty = False
                    break
                elif token in non_terminals:
                    before = len(FIRST[nt])
                    FIRST[nt] |= (FIRST[token] - {'ε'})

                    if len (FIRST[nt]) != before:
                        changed = True

                    if 'ε' not in FIRST[token]:
                        can_be_empty = False
                        break

            if can_be_empty:
                before = len(FIRST[nt])
                FIRST[nt].add('ε')

                if len(FIRST[nt]) != before:
                    changed = True

FIRST

{'<parent-expr>': {'(',
  '+',
  '-',
  '=',
  '?',
  'IDENTIFIER',
  'NUMBER',
  '×',
  'λ',
  '≜'},
 '<program>': {'(', 'IDENTIFIER', 'NUMBER'},
 '<expr>': {'(', 'IDENTIFIER', 'NUMBER'}}

In [5]:
for nt, s in FIRST.items():
    print(f'{nt}: {s}')

<parent-expr>: {'-', '≜', 'IDENTIFIER', '?', 'NUMBER', '(', '=', '×', 'λ', '+'}
<program>: {'(', 'IDENTIFIER', 'NUMBER'}
<expr>: {'(', 'NUMBER', 'IDENTIFIER'}


In [6]:
FOLLOW = {nt: set() for nt in non_terminals}
FOLLOW['<program>'].add('$')

changed = True
while changed:
    changed = False

    for head, productions in grammar.items():
        for body in productions:
            for i, B in enumerate(body):
                if B in non_terminals:
                    beta = body[i + 1:]

                    if beta:
                        first_beta = set()
                        can_be_empty = True

                        for sym in beta:
                            sym_first = FIRST[sym] if sym in FIRST else {sym}

                            first_beta |= (sym_first - {'ε'})
                            if 'ε' not in sym_first:
                                can_be_empty = False
                                break

                        if can_be_empty:
                            first_beta.add('ε')

                        before = len(FOLLOW[B])
                        FOLLOW[B] |= (first_beta - {'ε'})

                        if 'ε' in first_beta:
                            FOLLOW[B] |= FOLLOW[head]

                        if len(FOLLOW[B]) != before:
                            changed = True
                    else:
                        before = len(FOLLOW[B])
                        FOLLOW[B] |= FOLLOW[head]

                        if len(FOLLOW[B]) != before:
                            changed = True

FOLLOW


{'<parent-expr>': {')'},
 '<program>': {'$'},
 '<expr>': {'$', '(', ')', '<expr>*', 'IDENTIFIER', 'NUMBER'}}

In [7]:
for nt, s in FOLLOW.items():
    print(f'{nt}: {s}')

<parent-expr>: {')'}
<program>: {'$'}
<expr>: {')', 'IDENTIFIER', 'NUMBER', '$', '(', '<expr>*'}


In [8]:
parse_table = {nt: {} for nt in non_terminals}

conflicts = []

for A, productions in grammar.items():
    for alpha in productions:
        first_set = set()
        can_be_empty = True

        for sym in alpha:
            if sym in terminals:
                first_set.add(sym)
                can_be_empty = False
                break

            else:
                first_set |= (FIRST[sym] - {'ε'})
                if 'ε' not in FIRST[sym]:
                    can_be_empty = False
                    break

        if can_be_empty:
            first_set.add('ε')

        for a in first_set - {'ε'}:
            if a in parse_table[A]:
                conflicts.append((A, a, parse_table[A][a], alpha))

            parse_table[A][a] = alpha

        if 'ε' in first_set:
            for b in FOLLOW[A]:
                if b in parse_table[A]:
                    conflicts.append((A, b, parse_table[A][b], alpha))

                parse_table[A][b] = alpha

for nt in parse_table:
    print(f"\nNon-Terminal: {nt}")

    for t, rule in parse_table[nt].items():
        print(f"  M[{nt}, {t}] = {rule}")



Non-Terminal: <parent-expr>
  M[<parent-expr>, +] = ['+', '<expr>', '<expr>']
  M[<parent-expr>, ×] = ['×', '<expr>', '<expr>']
  M[<parent-expr>, =] = ['=', '<expr>', '<expr>']
  M[<parent-expr>, -] = ['-', '<expr>', '<expr>']
  M[<parent-expr>, ?] = ['?', '<expr>', '<expr>', '<expr>']
  M[<parent-expr>, λ] = ['λ', 'IDENTIFIER', '<expr>']
  M[<parent-expr>, ≜] = ['≜', 'IDENTIFIER', '<expr>', '<expr>']
  M[<parent-expr>, (] = ['<expr>', '<expr>*']
  M[<parent-expr>, NUMBER] = ['<expr>', '<expr>*']
  M[<parent-expr>, IDENTIFIER] = ['<expr>', '<expr>*']

Non-Terminal: <program>
  M[<program>, (] = ['<expr>']
  M[<program>, NUMBER] = ['<expr>']
  M[<program>, IDENTIFIER] = ['<expr>']

Non-Terminal: <expr>
  M[<expr>, NUMBER] = ['NUMBER']
  M[<expr>, IDENTIFIER] = ['IDENTIFIER']
  M[<expr>, (] = ['(', '<parent-expr>', ')']


In [9]:
!pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [10]:
import pandas as pd

all_terminals = set()

for entries in parse_table.values():
    all_terminals.update(entries.keys())

sorted_terminals = sorted(all_terminals)

table_data = {}

for terminal in sorted_terminals:
    column = []
    for nt in non_terminals:
        if terminal in parse_table[nt]:
            rule = parse_table[nt][terminal]

            formatted_rule = f"{nt} → {' '.join(rule)}"
            column.append(formatted_rule)
        else:
            column.append('')

    table_data[terminal] = column

df = pd.DataFrame(table_data, index=sorted(non_terminals))
df

,(,+,-,=,?,IDENTIFIER,NUMBER,×,λ,≜
<expr>,<parent-expr> → <expr> <expr>*,<parent-expr> → + <expr> <expr>,<parent-expr> → - <expr> <expr>,<parent-expr> → = <expr> <expr>,<parent-expr> → ? <expr> <expr> <expr>,<parent-expr> → <expr> <expr>*,<parent-expr> → <expr> <expr>*,<parent-expr> → × <expr> <expr>,<parent-expr> → λ IDENTIFIER <expr>,<parent-expr> → ≜ IDENTIFIER <expr> <expr>
<parent-expr>,<program> → <expr>,,,,,<program> → <expr>,<program> → <expr>,,,
<program>,<expr> → ( <parent-expr> ),,,,,<expr> → IDENTIFIER,<expr> → NUMBER,,,


In [11]:
if not conflicts:
    print('No conflicts - Each (non-terminal, lookahead) pair maps to only 1 production')
else:
    print('Conflicts found')
    for nt, term, rule1, rule2 in conflicts:
        print(f'  M[{nt}, {term}] has multiple entries:')
        print(f'- {rule1}')
        print(f'- {rule2}')

No conflicts - Each (non-terminal, lookahead) pair maps to only 1 production


### Grammar Properties
* **Unambiguous:** Each syntactically valid MiniLisp expression has only one parse tree.
* **Left-factored:** By using `<paren-expr>` inside parentheses, the grammar avoids having multiple productions that begin with `'('`.
* **If expanded directly:**
  Writing `<expr> ::= NUMBER | IDENTIFIER | '(' <expr> ')' | '(' '+' <expr> <expr> ')' | ...`
  would cause FIRST/FIRST conflicts for `'('`, breaking LL(1) compatibility.